In [ ]:
pip install --upgrade langchain   
pip install --upgrade nltk
pip install --upgrade openai   
pip install unstructured         
pip install pinecone-client
pip install tiktoken
pip install PyPDF2

In [8]:
import os

In [3]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.callbacks import get_openai_callback
from langchain.chains.question_answering import load_qa_chain

In [6]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

from nltk import word_tokenize,sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\monir\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [9]:
llm = OpenAI(temperature=0.9)

In [11]:
loader = UnstructuredPDFLoader("Cowan-en.pdf")

In [12]:
data = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\monir\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [13]:
print(f'there are {len(data)} document(s) in your data')
print(f'there are {len(data[0].page_content)} characters in the document')

there are 1 document(s) in your data
there are 81121 characters in the document


In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [15]:
print (f'Now have {len(texts)} documents')

Now have 87 documents


In [17]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

C:\Users\monir\AppData\Roaming\Python\Python311\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [18]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get('OPENAI_API_KEY'))

In [19]:
pinecone.init(
    api_key=os.environ.get("PINECONE_API_KEY"),
    environment="northamerica-northeast1-gcp"
)
index_name = "langchain2"

In [21]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [22]:
query = "What was the underlying crime the defendant was accused of assisting?"
docs = docsearch.similarity_search(query)

In [23]:
chain = load_qa_chain(llm, chain_type="stuff")

In [24]:
chain.run(input_documents=docs, question = query)

' The defendant was accused of assisting in armed robbery.'

In [15]:
# tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [16]:
# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [17]:
# text = "What are the five vacation destinations for someone that likes to eat kabab?"
# print(llm(text))

In [18]:
# prompt = PromptTemplate(
#     input_variables=["food"],
#     template="What are 3 vacation destinations for someone who likes to eat {food}?",
# )

In [19]:
# print(prompt.format(food="dessert"))

In [20]:
# print(llm(prompt.format(food="falafel")))

In [21]:
# chain = LLMChain(llm=llm, prompt=prompt)

In [22]:
# print(chain.run("pizza"))

In [23]:
# agent.run("Who is the current leader of Iran? Who are the family members of that individual?")